## Регрессия
### Цель
Задача — предсказать цену продажи каждого дома. Для каждого идентификатора в тестовом наборе вам необходимо предсказать значение переменной SalePrice. 

Заявки оцениваются по среднеквадратичной ошибке (RMSE)  между логарифмом прогнозируемого значения и логарифмом наблюдаемой цены продажи.

In [26]:
import pandas as pd
import joblib

In [27]:
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor

In [28]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [29]:
from helper.plots import feature_plots
from helper.utils import data_tools

### Описание набора данных
#### Описания файлов

* train.csv — обучающий набор
* test.csv — тестовый набор
* data_description.txt — полное описание каждого столбца, изначально подготовленное Дином Де Коком, но слегка отредактированное, чтобы соответствовать названиям столбцов, используемым здесь
* sample_submission.csv — эталонный отчет, полученный на основе линейной регрессии по году и месяцу продажи, площади участка и количеству спален.

#### Поля данных:

| Column Name     | Translation Name                                                  |
|-----------------|-------------------------------------------------------------------|
| SalePrice       |Цена продажи недвижимости в долларах                               | 
| MSSubClass      |Класс здания                                                       |
| MSZoning        |Общая классификация зонирования                                    |
| LotFrontage     |Линейные футы улицы, прилегающей к участку                         |
| LotArea         |Размер участка в квадратных футах                                  | 
| Street          |Тип подъездной дороги                                              |
| Alley           |Тип доступа к аллее                                                |
| LotShape        |Общая форма участка                                                |
| LandContour     |Ровность участка                                                   |
| Utilities       |Тип доступных коммунальных услуг                                   |
| LotConfig       |Конфигурация лота                                                  |
| LandSlope       |Уклон участка                                                      |
| Neighborhood    |Физические местоположения в пределах города Эймс                   |
| Condition1      |Близость к главной дороге или железной дороге                      |
| Condition2      |Близость к главной дороге или железной дороге (если есть вторая)   |
| BldgType        |Тип жилища                                                         |
| HouseStyle      | Стиль жилища                                                      |
| OverallQual     |Общее качество материалов и отделки                                |
| OverallCond     |Общая оценка состояния                                             |
| YearBuilt       |Первоначальная дата постройки                                      |
| YearRemodAdd    |Дата реконструкции                                                 |
| RoofStyle       |Тип крыши                                                          | 
| RoofMatl        |Кровельный материал                                                |
| Exterior1st     |Внешняя отделка дома                                               |
| Exterior2nd     |Внешняя отделка дома (если используется более одного материала)    |
| MasVnrType      |Тип облицовки каменной кладкой                                     |
| MasVnrArea      |Площадь облицовки каменной кладкой в ​​квадратных футах              |
| ExterQual       |Качество внешнего материала                                        |
| ExterCond       |Текущее состояние материала снаружи                                |
| Foundation      |Тип фундамента                                                     |
| BsmtQual        |Высота подвала                                                     |
| BsmtCond        |Общее состояние подвала                                            |
| BsmtExposure    |Стены подвала на уровне выхода или сада                            |
| BsmtFinType1    |Качество отделки подвала                                           |
| BsmtFinSF1      |Тип 1, готовые квадратные футы                                     |
| BsmtFinType2    |Качество второй обработанной области (если присутствует)           |
| BsmtFinSF2      |Тип 2, готовые квадратные футы                                     |
| BsmtUnfSF       |Неотделанная площадь подвала в квадратных футах                    |
| TotalBsmtSF     |Общая площадь подвала в квадратных футах                           |
| Heating         |Тип отопления                                                      |
| HeatingQC       |Качество и состояние отопления                                     |
| CentralAir      |Центральное кондиционирование воздуха                              |
| Electrical      |Электрическая система                                              |
| 1stFlrSF        |Площадь первого этажа (квадратные футы)                            |
| 2ndFlrSF        |Площадь второго этажа (квадратные футы)                            |
| LowQualFinSF    |Низкое качество отделки квадратных футов (все этажи)               |
| GrLivArea       |Жилая площадь над уровнем земли (квадратные футы)                  |
| BsmtFullBath    |Ванные комнаты в подвале                                           |
| BsmtHalfBath    |Полуванные в подвале                                               |
| FullBath        |Полноценные ванные комнаты выше уровня земли                       |
| HalfBath        |Полуванны выше уровня земли                                        |
| Bedroom         |Количество спален над цокольным этажом                             |
| Kitchen         |Количество кухонь                                                  |
| KitchenQual     |Качество кухни                                                     |
| TotRmsAbvGrd    |Общее количество комнат выше уровня земли (без учета ванных комнат)|
| Functional      |Оценка функциональности дома                                       |
| Fireplaces      |Количество каминов                                                 |
| FireplaceQu     |Качество камина                                                    |
| GarageType      |Расположение гаража                                                |
| GarageYrBlt     |Год постройки гаража                                               |
| GarageFinish    |Внутренняя отделка гаража                                          |
| GarageCars      |Размер гаража в зависимости от вместимости автомобиля              |
| GarageArea      |Площадь гаража в квадратных футах                                  |
| GarageQual      |Качество гаража                                                    |
| GarageCond      |Состояние гаража                                                   |
| PavedDrive      |Асфальтированная подъездная дорога                                 |
| WoodDeckSF      |Площадь деревянной террасы в квадратных футах                      |
| OpenPorchSF     |Площадь открытой веранды в квадратных футах                        |
| EnclosedPorch   |Площадь закрытой веранды в квадратных футах                        |
| 3SsnPorch       |Площадь крыльца, рассчитанного на три сезона, в квадратных футах   |
| ScreenPorch     |Площадь веранды в квадратных футах                                 |
| PoolArea        |Площадь бассейна в квадратных футах                                |
| PoolQC          |Качество бассейна                                                  |
| Fence           |Качество забора                                                    |
| MiscFeature     |Разные функции, не охваченные другими категориями                  |
| MiscVal         |$Value разного признака                                            |
| MoSold          |Месяц продаж                                                       |
| YrSold          |Год продажи                                                        |
| SaleType        |Тип продажи                                                        |
| SaleCondition   |Состояние продажи                                                  |

Зафиксируем наши испытания, создав переменную для **random_state**.

In [30]:
SEED = 42

### 1. Загрузка данных

Загрузим датасет, который представлен в виде **train** и **test**.

In [31]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

print(f'Размер train: {train.shape}')
print(f'Размер test: {test.shape}')

Размер train: (1460, 81)
Размер test: (1459, 80)


Заметим, что **отсутствует некий столбец в test** и предварительно можно оценить что это `SalePrice` - целевая переменная.

In [32]:
missing_columns = set(train.columns) - set(test.columns)
missing_columns = ", ".join(missing_columns)
print(f'Отсутствующий столбец: {missing_columns}')

Отсутствующий столбец: SalePrice


Т.к. **в тесте отсутствует целевая переменная**, то эти данные можно использовать только в kaggle, но при измении трейна необходимо так же изменять и тест иначе невозможно получить предикты по разным фичам.

Предварительно посмотрим на данные:

In [33]:
train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [34]:
train.tail(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

Максимальное количество данных 1460, наблюдаем **большое количество пропущенных значений**, которые надо преобработать, при этом **первично можно оценить, что тип данных указан правильно**. Есть **фичи с сильными повреждениями в данных**, поэтому возможно придется более детально их рассмотреть.

Перейдем к EDA.

### 2. Разведочный анализ данных (EDA)

#### 2.1. Проверка на дубликаты

Необходимо **проверить данные на дубликаты**, но скорее всего они отсутствуют, т.к. проверяем только 7 строк (по минимальному в датасете).

In [36]:
print(f'Количество дубликатов в данных: {train.duplicated().sum()}')

Количество дубликатов в данных: 0


**Исключим фичи** с редкими данными и с уникальными значениями.

In [37]:
neg_columns = ['PoolQC', 'MiscFeature', 'Fence','FireplaceQu', 'Id']
duplic_columns = [col for col in train.columns if col not in neg_columns]

In [38]:
print(f'Количество дубликатов в данных: {train[duplic_columns].duplicated().sum()}')

Количество дубликатов в данных: 0


Теперь с полной уверенность можно сказать, что **дубликаты отсутствуют**. Далее необходимо **проверить пропущенные значения**.

#### 2.2. Проверка на пропущенные значения

Проверим **количество и долю пропущенных значений.**

In [39]:
missing_value = data_tools.DataPreprocessor(train).get_missing()
miss_ratio = data_tools.DataPreprocessor(train).get_missing_ratio()

missing = pd.concat([missing_value , miss_ratio], axis=1)
missing.columns = ['count', 'ratio']

display(missing)

,count,ratio
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
MasVnrType,872,0.597260
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479


**Топ 7 фичей с пропусков более 10%**, поэтому стоит обратить на них особое внимание. Заметим, что фичи отвечающие за гараж имеют одинаковое количество пропусков, что может указывать на отсутствие гаража. У подвала тоже количество пропусков одинаковое, и возможно, по той же причине - нет подвала в доме.

##### 2.3. Оценка типов данных

Выясним какие столбцы с категориальными переменными, а какие с количественными.

In [40]:
numeric_columns_eda = train.select_dtypes(include=['int', 'float']).columns.to_list()
categorical_columns_eda = train.select_dtypes(exclude=['int', 'float']).columns.to_list()

**Рассмотрим** детальнее **категориальные фичи**.

In [41]:
train[categorical_columns_eda].describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


##### Краткая характеристика категориальных переменных:

| Column Name   | Translation Name      | Column Result                                                                                                                   |
|---------------|-------------|--------------------------------------------------------------------------------------------------------------------------|
| MSZoning       |Общая классификация зонирования |Данные в норме.|
| Street     |Тип подъездной дороги |В данных количество Pave 1454 из 1460, поэтому можно удалить этот столбец.|
| Alley    |Тип доступа к аллее |Данных сильно не хватает, но часть можно попробовать восстановить.|
| LotShape   |Общая форма участка | Данные в норме.|
| LandContour     |Общая оценка состояния | Данные в норме.|
| Utilities       |Тип доступных коммунальных услуг | В данных количество AllPub 1459 из 1460, поэтому можно удалить этот столбец.|
| LotConfig     |Конфигурация лота | Данные в норме.|
| LandSlope    |Уклон участка | Данные в норме.|
| Neighborhood   |Физические местоположения в пределах города Эймс | Данные в норме.|
| Condition1     |Близость к главной дороге или железной дороге | Данные в норме.|
| Condition2       |Близость к главной дороге или железной дороге (если есть вторая) | Данные в норме.|
| BldgType     |Тип жилища | Данные в норме.|
| HouseStyle    |Стиль жилища | Данные в норме.|
| RoofStyle   |Тип крыши | Данные в норме.|
| RoofMatl     |Кровельный материал | Данные в норме.|
| Exterior1st       |Внешняя отделка дома | Данные в норме.|
| Exterior2nd     |Внешняя отделка дома (если используется более одного материала) | Данные в норме.|
| MasVnrType   |Тип облицовки каменной кладкой | Данные в норме. Можно восстановить.|
| ExterQual     |Качество внешнего материала | Данные в норме.|
| ExterCond     |Текущее состояние материала снаружи | Данные в норме.|
| Foundation     |Тип фундамента | Данные в норме.|
| BsmtQual     |Высота подвала | Данные в норме. Можно восстановить.|
| BsmtCond     |Общее количество комнат выше уровня земли | Данные в норме. Можно восстановить.|
| BsmtExposure     |Стены подвала на уровне выхода или сада | Данные в норме. Можно восстановить.|
| BsmtFinType1     |Качество отделки подвала | Данные в норме. Можно восстановить.|
| BsmtFinType2     |Качество второй обработанной области (если присутствует) |Данные в норме. Можно восстановить.|
| Heating     |Тип отопления |Данные в норме.|
| HeatingQC     |Качество и состояние отопления |Данные в норме.|
| CentralAir     |Центральное кондиционирование воздуха |Данные в норме.|
| Electrical     |Электрическая система |Данные в норме, но есть один пропуск.|
| KitchenQual     |Качество кухни | Данные в норме.|
| Functional     |Оценка функциональности дома | Данные в норме.|
| FireplaceQu     |Качество камина |Данные в норме. Можно восстановить.|
| GarageType     |Расположение гаража | Данные в норме. Можно восстановить.|
| GarageFinish     |Внутренняя отделка гаража | Данные в норме. Можно восстановить.|
| GarageQual     |Качество гаража | Данные в норме. Можно восстановить.|
| GarageCond     |Состояние гаража | Данные в норме. Можно восстановить.|
| PavedDrive     |Асфальтированная подъездная дорога | Данные в норме.|
| Fence     |Качество забора | Данных сильно не хватает, но скорее всего пропуски - отсутствие забора.|
| SaleType     |Тип продажи | Данные в норме.|
| SaleCondition     |Состояние продажи | Данные в норме.|

Рассмотрим детальнее количественные фичи.

In [42]:
train[numeric_columns_eda].describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


##### Краткая характеристика количественных переменных:

| Column Name   | Translation Name      | Column Result                                                                                                                   |
|---------------|-------------|--------------------------------------------------------------------------------------------------------------------------|
| MSSubClass       |Класс здания | В целом выглядит нормально, но стоит проверить данные на максимуме, т.к. наблюдается большое расхождение между 75% и max.|
| LotFrontage     |Линейные футы улицы, прилегающей к участку | Так же как и в классе здания надо проверить расхождение между 75% и max.|
| LotArea    |Размер участка в квадратных футах | Разница просто невероятная между 75% и max, что свидетельствует о возможном выбросе.|
| OverallQual   |Общее качество материалов и отделки | Данные в норме.|
| OverallCond     |Общая оценка состояния | Данные в норме.|
| YearBuilt       |Первоначальная дата постройки | Данные в норме.|
| YearRemodAdd     |Дата реконструкции | Данные в норме, но стоит проверить, чтобы дата реконструкции не была меньше, чем год постройки.|
| MasVnrArea    |Площадь облицовки каменной кладкой в ​​квадратных футах | Стоит проверить данные на максимуме, т.к. наблюдается большое расхождение между 75% и max.|
| BsmtFinSF1   |Качество отделки подвала | Стоит проверить данные на максимуме, т.к. наблюдается большое расхождение между 75% и max.|
| BsmtFinSF2     |Качество второй обработанной области | Стоит проверить данные на максимуме, т.к. наблюдается большое расхождение между 75% и max.|
| BsmtUnfSF       |Неотделанная площадь подвала в квадратных футах | Данные в норме.|
| TotalBsmtSF     |Общая площадь подвала в квадратных футах | Данные в норме.|
| 1stFlrSF    |Площадь первого этажа | Данные в норме.|
| 2ndFlrSF   |Качество второй обработанной области | Данные в норме.|
| LowQualFinSF     |Низкое качество отделки квадратных футов | Низкое качество отделки квадратных футов - Стоит проверить данные на максимуме, т.к. наблюдается большое расхождение между 75% и max.|
| GrLivArea       |Жилая площадь над уровнем земли | Данные в норме.|
| BsmtFullBath     |Ванные комнаты в подвале | В max видно 3 ванны в подвале, что можно поставить под сомнение в истинности данных.|
| BsmtHalfBath   |Полуванные в подвале | Данные в норме.|
| FullBath     |Полноценные ванные комнаты выше уровня земли | Возникает избыточных в этих, поэтому стоит от них избавиться.|
| HalfBath     |Полуванны выше уровня земли | Так же избавиться от данных, т.к. они избыточны.|
| BedroomAbvGr     | | Данные в норме.|
| KitchenAbvGr     | | Данные в норме.|
| TotRmsAbvGrd     |Общее количество комнат выше уровня земли | Данные в норме.|
| Fireplaces     |Количество каминов | Данные в норме.|
| GarageYrBlt     |Год постройки гаража | Данные в норме.|
| GarageCars     |Размер гаража в зависимости от вместимости автомобиля |Данные в норме.|
| GarageArea     |Площадь гаража в квадратных футах |Данные в норме.|
| WoodDeckSF     |Площадь деревянной террасы в квадратных футах |Данные в норме.|
| OpenPorchSF     |Площадь открытой веранды в квадратных футах |Данные в норме.|
| EnclosedPorch     |Площадь закрытой веранды в квадратных футах |Стоит проверить эти данные т.к. большинство значений в 0.|
| 3SsnPorch     |Площадь крыльца, рассчитанного на три сезона, в квадратных футах | Так же рассмотрим эти данные, т.к большинство значений в 0.|
| ScreenPorch     |Площадь веранды в квадратных футах | Так же большинство значений в 0.|
| PoolArea     |Площадь бассейна в квадратных футах |Так же большинство значений в 0.|
| MiscVal     |$Value разного признака | Этот столбец не несет в себе полезную информацию.|
| MoSold     |Месяц продаж | Данные в норме.|
| YrSold     |Год продажи | Данные в норме.|


### 3. Baseline

Для того чтобы понять, как **обрабатывать пропуски над фичами**, необходимо **оценивать действия** над ними. Для оценки действий обучим начальную модель и после будем смотреть **прирост по МАЕ**. 

In [43]:
baseline_dummy = data_tools.PipelineManager(train, DummyRegressor()).get_baseline()

In [44]:
print(f'МАЕ dummy: {baseline_dummy}')
print(f'Среднее значение: {round((train['SalePrice'].mean()), 2)}')
print(f'Относительная ошибка: {round((baseline_dummy / train['SalePrice'].mean()), 2) * 100}%')

МАЕ dummy: 59931.24
Среднее значение: 180921.2
Относительная ошибка: 33.0%


**По Dummy сложно** будет **оценивать действия**, т.к. **модель ошибается на 35%**, обучим более сильную модель, но без подбора гиперпараметров.

In [45]:
baseline_model = RandomForestRegressor(random_state=SEED)

In [46]:
baseline_forest = data_tools.PipelineManager(train, baseline_model).get_baseline()

In [47]:
print(f'МАЕ dummy: {baseline_forest}')
print(f'Среднее значение: {round((train['SalePrice'].mean()), 2)}')
print(f'Относительная ошибка: {round((baseline_forest / train['SalePrice'].mean()), 2) * 100}%')

МАЕ dummy: 17586.45
Среднее значение: 180921.2
Относительная ошибка: 10.0%


Относительная ошибка случайного леса гораздо меньше чем у дамми модели. По модели случайного леса можно оценивать прирост метрики. Сохраним модель для дальнейшей оценки.

In [48]:
joblib.dump(baseline_model, './model/baseline_model')

['./model/baseline_model']

In [49]:
path = './model/baseline_mae.json'
data_tools.save_json(baseline_forest, path)

✅ Данные сохранены в ./model/baseline_mae.json


In [50]:
missing.to_json('./model/missing.json', orient='split', force_ascii=False, indent=2)

#### Вывод

Датасет представлен двумя наборами данных - трейном и тестом. Наличие целевой метрики в трейне предполагает основную работу нам ним, а так же его разделение на обучающую и валидационную выборки для оценки метрики. В данных очень много пропусков, для устранения которые, возможно, потребуется более детальный анализ. Основная метрика качества это МАЕ, и результирующее МАЕ не должно превышать 18000 (не выше бэйзлайна), и по относительной ошибке не выше 10%. 